In [1]:
import os

os.chdir('../')

In [2]:
%pwd

'd:\\Python Project\\TextSummarize-NLP'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: float
    gradient_accumulation_steps: int

In [4]:
from src.textSummarizer.constants import *
from src.textSummarizer.utils.common import read_yaml, create_directories

In [5]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
        
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.TrainingArguments
        
        create_directories([config.root_dir])
        
        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_ckpt=config.model_ckpt,
            
            num_train_epochs=params.num_train_epochs,
            warmup_steps=params.warmup_steps,
            per_device_train_batch_size=params.per_device_train_batch_size,
            weight_decay=params.weight_decay,
            logging_steps=params.logging_steps,
            evaluation_strategy=params.evaluation_strategy,
            eval_steps=params.eval_steps,
            save_steps=params.save_steps,
            gradient_accumulation_steps=params.gradient_accumulation_steps
        )
        
        return model_trainer_config

In [6]:
from transformers import TrainingArguments, Trainer, \
DataCollatorForSeq2Seq, \
AutoModelForSeq2SeqLM, AutoTokenizer

from datasets import load_dataset, load_from_disk

import torch


c:\Users\qhuy9\anaconda3\envs\textSumma\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2024-03-18 21:49:15,003] INFO: config: PyTorch version 2.2.1 available.


In [7]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config
        
    def train(self):
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
        
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)
        
        # Loading Data
        dataset_samsum_pt = load_from_disk(self.config.data_path)
        
        trainer_args = TrainingArguments(
            output_dir=self.config.root_dir, num_train_epochs=1, warmup_steps=500,
            per_device_train_batch_size=1, per_device_eval_batch_size=1,
            weight_decay=0.01, logging_steps=10,
            evaluation_strategy='steps', eval_steps=500, save_steps=1e6,
            gradient_accumulation_steps=16
        ) 
        
        trainer = Trainer(model=model_pegasus, args=trainer_args,
                          tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                          train_dataset=dataset_samsum_pt['train'],
                          eval_dataset=dataset_samsum_pt['validation'])
        
        trainer.train()
        
        # Save model
        model_pegasus.save_pretrained(os.path.join(self.config.root_dir, 'pegasus-samsum-model'))
        
        # Save tokenizer
        tokenizer.save_pretrained(os.path.join(self.config.root_dir, 'tokenizer'))

In [8]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(config=model_trainer_config)
    model_trainer.train()
    
except Exception as e:
    raise e

[2024-03-18 21:49:15,454] INFO: common: yaml file: D:\Python Project\TextSummarize-NLP\config\config.yaml loaded successfully
[2024-03-18 21:49:15,457] INFO: common: yaml file: D:\Python Project\TextSummarize-NLP\params.yaml loaded successfully
[2024-03-18 21:49:15,459] INFO: common: created directory at: D:/Python Project/TextSummarize-NLP/artifacts
[2024-03-18 21:49:15,461] INFO: common: created directory at: D:/Python Project/TextSummarize-NLP/artifacts/model_trainer


Error while downloading from https://cdn-lfs.huggingface.co/google/pegasus-cnn_dailymail/cf34e28157605d3ced3bd184820ea2a9d9a30b69f4bb1df4aaa4145d0850290e?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27pytorch_model.bin%3B+filename%3D%22pytorch_model.bin%22%3B&response-content-type=application%2Foctet-stream&Expires=1711032561&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcxMTAzMjU2MX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9nb29nbGUvcGVnYXN1cy1jbm5fZGFpbHltYWlsL2NmMzRlMjgxNTc2MDVkM2NlZDNiZDE4NDgyMGVhMmE5ZDlhMzBiNjlmNGJiMWRmNGFhYTQxNDVkMDg1MDI5MGU%7EcmVzcG9uc2UtY29udGVudC1kaXNwb3NpdGlvbj0qJnJlc3BvbnNlLWNvbnRlbnQtdHlwZT0qIn1dfQ__&Signature=OzRxzYb1DIvcCdNLyU5j3%7EJVr6C9UAkc7A2fl4IaRE4jTMQi5XDh3j%7E2Emx9cA4U0A9RHxVPV1jhOk8N2QG9OtYYCdIX5gKHHF-VXYMbamvNVITmC9cKC5DbLH%7Ed%7Eyx6%7EuQCbeyfmk7DYuVMsn%7EEwOPwzD6aHz89veK96x-VDgV-BVlELJi2M5qOzubiO8o03%7EJFJ7OTHxoVSjxuYVzT-ocYEMCMPs89TNbDD0ngGI%7EVeUdSNENDRuBBMuTRWLc7E

[2024-03-18 21:58:39,446] WARNING: file_download: Error while downloading from https://cdn-lfs.huggingface.co/google/pegasus-cnn_dailymail/cf34e28157605d3ced3bd184820ea2a9d9a30b69f4bb1df4aaa4145d0850290e?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27pytorch_model.bin%3B+filename%3D%22pytorch_model.bin%22%3B&response-content-type=application%2Foctet-stream&Expires=1711032561&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcxMTAzMjU2MX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9nb29nbGUvcGVnYXN1cy1jbm5fZGFpbHltYWlsL2NmMzRlMjgxNTc2MDVkM2NlZDNiZDE4NDgyMGVhMmE5ZDlhMzBiNjlmNGJiMWRmNGFhYTQxNDVkMDg1MDI5MGU%7EcmVzcG9uc2UtY29udGVudC1kaXNwb3NpdGlvbj0qJnJlc3BvbnNlLWNvbnRlbnQtdHlwZT0qIn1dfQ__&Signature=OzRxzYb1DIvcCdNLyU5j3%7EJVr6C9UAkc7A2fl4IaRE4jTMQi5XDh3j%7E2Emx9cA4U0A9RHxVPV1jhOk8N2QG9OtYYCdIX5gKHHF-VXYMbamvNVITmC9cKC5DbLH%7Ed%7Eyx6%7EuQCbeyfmk7DYuVMsn%7EEwOPwzD6aHz89veK96x-VDgV-BVlELJi2M5qOzubiO8o03%7EJFJ7OTHxoVSjxuY

ConnectionError: (MaxRetryError('HTTPSConnectionPool(host=\'cdn-lfs.huggingface.co\', port=443): Max retries exceeded with url: /google/pegasus-cnn_dailymail/cf34e28157605d3ced3bd184820ea2a9d9a30b69f4bb1df4aaa4145d0850290e?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27pytorch_model.bin%3B+filename%3D%22pytorch_model.bin%22%3B&response-content-type=application%2Foctet-stream&Expires=1711032561&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcxMTAzMjU2MX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9nb29nbGUvcGVnYXN1cy1jbm5fZGFpbHltYWlsL2NmMzRlMjgxNTc2MDVkM2NlZDNiZDE4NDgyMGVhMmE5ZDlhMzBiNjlmNGJiMWRmNGFhYTQxNDVkMDg1MDI5MGU~cmVzcG9uc2UtY29udGVudC1kaXNwb3NpdGlvbj0qJnJlc3BvbnNlLWNvbnRlbnQtdHlwZT0qIn1dfQ__&Signature=OzRxzYb1DIvcCdNLyU5j3~JVr6C9UAkc7A2fl4IaRE4jTMQi5XDh3j~2Emx9cA4U0A9RHxVPV1jhOk8N2QG9OtYYCdIX5gKHHF-VXYMbamvNVITmC9cKC5DbLH~d~yx6~uQCbeyfmk7DYuVMsn~EwOPwzD6aHz89veK96x-VDgV-BVlELJi2M5qOzubiO8o03~JFJ7OTHxoVSj